# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? 

**Answer:** This is a classification problem since we're expecting two discrete states for the target variable -- pass or fail. If we wanted to predict the exact percentage grade a student might achieve, then that would be an example of a regression problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
# Subtract 1 since one column includes the results (aka target), which isn't a feature.
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = float(n_passed)/n_students*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

# I used skikit-learn's train_test_split utility which automatically shuffles and samples.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=1)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

----------------------------------------------------

I selected three supervised learning models: support vector machines (SVM), nearest neighbors and decision trees. 

### Support vector machines
This algorithm is often used in classification problems to create linear hyperplanes between samples. Its strength is that it tries to maximize the margin between the samples, which allows it to be more robust when it sees new samples. Its disadvantage is that it draws lines and as such doesn't have as many degrees of freedom for creating complex boundaries when compared to nearest neighbors or random forests.

I chose this model to try out since it is a popular algorithm (especially when combined with kernel methods) to classify samples. It is good for creating generalized models that aren't as subject to overfitting.

| Training set size         | 100   | 200   | 300   |
|---------------------------|-------|-------|-------|
| Training time (secs)      | 0.002 | 0.006 | 0.012 |
| Prediction time (secs)    | 0.002 | 0.005 | 0.008 |
| F1 score for training set | 0.859 | 0.858 | 0.858 |
| F1 score for test set     | 0.833 | 0.841 | 0.846 |

### K-nearest neighbors
K-nearest neighbors is used in classification problems and memorizes the data. It uses _k_ of the closest samples to a point and performs a majority vote to classify the new point.

Its advantages includes its simplicity and ability to classify samples that have complex boundaries between the different types. Its disadvantages include being computationally expensive for large datasets (since it memorizes data) and it doesn't create a generalized model that can be interpreted.

| Training set size         | 100   | 200   | 300   |
|---------------------------|-------|-------|-------|
| Training time (secs)      | 0.001 | 0.001 | 0.001 |
| Prediction time (secs)    | 0.002 | 0.004 | 0.012 |
| F1 score for training set | 0.788 | 0.834 | 0.856 |
| F1 score for test set     | 0.772 | 0.797 | 0.768 |

I chose this model since (being a student intervention system) similar students may demonstrate the same signs for needing support.

### Random forests
Random forests are again used for classification problems, particularly when it's difficult to draw simple lines between the boundaries of samples. It's strength is its many possible degrees of freedom to draw boundaries, but this is also its weakness, as one can overfit the data. This means that it can make it hard to generalize enough for new, unseen samples.

Random forests can be considered comprised of decision trees. It is an "ensemble" method.

I chose this model to try out since there are many variables in the dataset, so using random forests may be a good fit for data of high complexity.

| Training set size         | 100   | 200   | 300   |
|---------------------------|-------|-------|-------|
| Training time (secs)      | 0.012 | 0.012 | 0.010 |
| Prediction time (secs)    | 0.001 | 0.001 | 0.001 |
| F1 score for training set | 0.956 | 0.960 | 0.960 |
| F1 score for test set     | 0.739 | 0.800 | 0.770 |


In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.013
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.011
F1 score for training set: 0.858387799564


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.846153846154


In [14]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

# Notes: I would actually probably have used sklearn's libraries to do this. However, I'll
# make a loop that uses training set sizes of 100, 200 and 300 per the instructions.
rows = [100, 200, 300]

for r in rows:
    # Train on a subset and predict on the original test set per the instructions.
    train_predict(clf, X_train.iloc[0:r], y_train.iloc[0:r], X_test, y_test)

------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.859060402685
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.833333333333
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.858064516129
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.84076433121
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.012
Predicting labels using SVC...
Done!
Prediction time (secs): 0.009
F1 score for training set: 0.858387799564
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.846153846154


In [15]:
## Random forests

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=3)

rows = [100, 200, 300]

for r in rows:
    # Train on a subset and predict on the original test set per the instructions.
    train_predict(clf, X_train.iloc[0:r], y_train.iloc[0:r], X_test, y_test)

------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.015
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.955223880597
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.742857142857
------------------------------------------
Training set size: 200
Training RandomForestClassifier...
Done!
Training time (secs): 0.012
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.936329588015
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.738461538462
------------------------------------------
Training set size: 300
Training RandomForestClassifier...
Done!
Training time (secs): 0.010
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 sc

In [16]:
## K-nearest neighbors

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

rows = [100, 200, 300]

for r in rows:
    # Train on a subset and predict on the original test set per the instructions.
    train_predict(clf, X_train.iloc[0:r], y_train.iloc[0:r], X_test, y_test)

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.788321167883
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.772727272727
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.834482758621
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.797101449275
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.010
F1 score for training

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

-------------------

### Selected Model
I selected the support vector machines (SVM) algorithm when compared to both random forests and k-nearest neighbors. The SVM algorithm has the best performance when dealing with unseen data -- it has the highest F1 score on the test data set.

While it does have the highest computational cost for training time (12 ms with 300 students) among the three models, the higher performance on F1 score is worth a few milliseconds since this isn't a real time application (like a self-driving automobile) with demanding time constraints.

### Model Theory of Operation
If you put all of the student data on a multi-dimensional scatterplot there are linear boundaries that you can imagine between at-risk students and all other students. Support vector machines attempts to find the optimal separating lines between these two groups by drawing lines (or planes) between the data.

Since there are many such possible separating borders that can be drawn, SVM selects lines that have the most amount of margin (or padding) between the two types of students. This allows SVM to better generalize and predict student performance with data it hasn't seen previously.

### Fine Tuning
I used the entire training set with GridSearchCV, which iterates over many parameters and finds the parameters achieving the best score (in this case F1).

One difference between the original results and when using GridSearch is that the latter also does cross-validation as part of its process.

GridSearch ultimately selected a C value (penalty parameter) of 1.0, which is actually the default. It also selected a gamma (kernel coefficient) of 0.06, which is slightly different than the default of 1/n_features (1/30).

The final F1 score is 0.82 for the training set and 0.75 for the test set.


In [49]:
# TODO: Fine-tune your model and report the best F1 score

from sklearn.grid_search import GridSearchCV

# Make an f1 scorer for Grid Search. A custom function was used above.
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, pos_label="yes")

# Create the model
clf = svm.SVC()

# Parameters to sweep
C_range = [0.1, 0.5, 1.0, 10, 10.0]
g_range = [0.01, 0.05, 0.06, 0.1, 0.2, 1.0]
param_grid = [{'C': C_range, 'gamma': g_range}]

# Search for best parameters
gs = GridSearchCV(clf, param_grid, scoring=f1_scorer, cv=10)
gs.fit(X_train, y_train)

# Show the best results
print('F1 Score on Training Data: ' + str(gs.best_score_))
print(gs.best_params_)

# See results on test dataset
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print('F1 Score on Test Data: ' + str(clf.score(X_test, y_test)))

F1 Score on Training Data: 0.816794959735
{'C': 1.0, 'gamma': 0.06}
F1 Score on Test Data: 0.747368421053


In [64]:
"""Count words."""

def count_words(s, n):
    """Return the n most frequently occuring words in s."""
    
    # TODO: Count the number of occurences of each word in s
    s = s.split()
    
    freq = []
    
    for w in s:
        freq.append(s.count(w))
        
    results = list(set(zip(s,freq)))
    
        
    top_n = sorted(results, key=lambda x: x[1], reverse=True)[0:n]
    # TODO: Sort the occurences in descending order (alphabetically in case of ties)
    
    # TODO: Return the top n words as a list of tuples (<word>, <count>)
    return top_n


def test_run():
    """Test count_words() with some inputs."""
    print count_words("cat bat mat cat bat cat", 3)
    print count_words("betty bought a bit of butter but the butter was bitter", 3)
    
test_run()

[('cat', 3), ('bat', 2), ('mat', 1)]
[('butter', 2), ('a', 1), ('of', 1)]
